In [1]:
import robin_stocks as r
import robin_stocks.authentication as authentication
import robin_stocks.helper as helper
import robin_stocks.urls as urls

import pandas as pd
import os

In [2]:
class RH_Client:
    
    def __init__(self):
        self.data = None
        self.payload = None
        self.url = None
    
    def login(self, username, password, expiresIn=86400, scope='internal', by_sms=True, store_session=True, mfa_code=None):
        device_token = authentication.generate_device_token()

        # Challenge type is used if not logging in with two-factor authentication.
        if by_sms:
            challenge_type = "sms"
        else:
            challenge_type = "email"

        self.url = urls.login_url()
        self.payload = {
            'client_id': 'c82SH0WZOsabOXGP2sxqcj34FxkvfnWRZBKlBjFS',
            'expires_in': expiresIn,
            'grant_type': 'password',
            'password': password,
            'scope': scope,
            'username': username,
            'challenge_type': challenge_type,
            'device_token': device_token
        }
        self.data = helper.request_post(self.url, self.payload)
        return (self.data)
    
    def respond_challenge(self, mfa_token=None, sms_code=None):

        if mfa_token:
            self.payload['mfa_code'] = mfa_token

        # Handle case where mfa or challenge is required.
        if self.data:
            if 'mfa_required' in self.data:
                res = helper.request_post(self.url, self.payload, jsonify_data=False)
                self.data = res.json()
            elif 'challenge' in self.data:
                challenge_id = self.data['challenge']['id']
                res = respond_to_challenge(challenge_id, sms_code)
                helper.update_session(
                    'X-ROBINHOOD-CHALLENGE-RESPONSE-ID', challenge_id)
                self.data = helper.request_post(self.url, self.payload)
            # Update Session data with authorization or raise exception with the information present in data.
            if 'access_token' in self.data:
                token = '{0} {1}'.format(self.data['token_type'], self.data['access_token'])
                helper.update_session('Authorization', token)
                helper.set_login_state(True)
                self.data['detail'] = "logged in with brand new authentication code."
            else:
                raise Exception(self.data['detail'])
        else:
            raise Exception('Error: Trouble connecting to robinhood API. Check internet connection.')

In [3]:
username = os.environ['RH_USERNAME']
password = os.environ['PASSWORD']

client = RH_Client()
client.login(username, password, store_session=False)

{'mfa_required': True, 'mfa_type': 'sms'}

In [4]:
client.respond_challenge('794027')

In [5]:
# fetch orders
orders = r.get_all_stock_orders()

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...


In [6]:
clean_orders = []
for order in orders:
    filled = float(order['cumulative_quantity'])
    if len(order['executions']) > 0:
        instrument = r.helper.request_get(order['instrument'])
        symbol = instrument['symbol']
        side = order['side']
        fees = order['fees']
        price = order['average_price']
#         quantity = order['quantity'] # intented quantity for order
        quantity = order['cumulative_quantity'] # filled quantity
        timestamp = order['created_at']
        clean_orders.append({
            'Timestamp': timestamp,
            'Symbol': symbol,
            'Quantity': quantity,
            'Price': price,
            'Side': side,
            'Commission': 0.0,
            'Fee': fees,
            'Type': 'SHARE'
        })
#         print(f'Symbol: {symbol}, Side: {side}, Fees: {fees}, Price: {price}, Quantity: {quantity}, Timestamp: {timestamp}')

In [7]:
rh_executions = pd.DataFrame(clean_orders, columns=['Timestamp','Symbol', 'Quantity', 'Price', 'Side', 'Commission', 'Fee', 'Type'])
rh_executions.to_csv('rh_executions.csv', index=False)

display(rh_executions)

,Timestamp,Symbol,Quantity,Price,Side,Commission,Fee,Type
0,2020-10-09T18:08:52.469537Z,VVPR,1001.00000000,16.85000000,sell,0.0,0.50,SHARE
1,2020-10-09T18:06:53.461110Z,VVPR,1.00000000,17.09000000,buy,0.0,0.00,SHARE
2,2020-10-09T18:06:18.557620Z,VVPR,1000.00000000,16.80000000,buy,0.0,0.00,SHARE
3,2020-10-09T17:56:56.354543Z,VVPR,800.00000000,17.85010000,sell,0.0,0.42,SHARE
4,2020-10-09T17:49:55.369129Z,VVPR,800.00000000,21.29630000,buy,0.0,0.00,SHARE
5,2020-10-09T17:49:08.236745Z,VVPR,500.00000000,21.60000000,sell,0.0,0.30,SHARE
6,2020-10-09T17:48:02.614956Z,VVPR,500.00000000,21.50000000,buy,0.0,0.00,SHARE
7,2020-10-09T17:38:41.609082Z,VVPR,120.00000000,20.02000000,sell,0.0,0.07,SHARE
8,2020-10-09T17:38:07.901725Z,VVPR,120.00000000,19.50000000,buy,0.0,0.00,SHARE
9,2020-10-09T17:32:58.135046Z,VVPR,1000.00000000,18.70000000,sell,0.0,0.54,SHARE


In [8]:
r.logout()